In [1]:
from dataclasses import dataclass
from enum import StrEnum
from datetime import datetime, timedelta
import polars as pl
from common.constants.column_types import (
    CPZP_SCHEMA,
    OZP_SCHEMA,
    POHLAVI_CPZP,
    TYP_UDALOSTI,
)
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS, CPZP_COLUMNS
import pickle
from common.constants.objects import (
    Person,
    Gender,
    AgeCohort,
    Prescription,
    PrescriptionType,
)
from common.utils import draw_chart

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(60)


with open("./DATACON_data/cpzp_persons.pkl", "rb") as f:
    cpzp_persons: list[Person] = pickle.load(f)

with open("./DATACON_data/ozp_persons.pkl", "rb") as f:
    ozp_persons: list[Person] = pickle.load(f)

In [ ]:
persons = cpzp_persons

vax_persons = [p for p in persons if p.vaccines]
print(f"Počet osob s očkováním: {len(vax_persons)}")

no_vax_no_prescriptions = [p for p in persons if not p.vaccines and not p.prescriptions]
print(f"Počet osob bez očkování a bez předpisů: {len(no_vax_no_prescriptions)}")


prescriptions_for_30_50 = [
    1
    for p in persons
    for prescription in p.prescriptions
    if prescription.age_cohort_at_prescription == AgeCohort.BETWEEN_30_AND_50
]
print(f"Počet předpisů pro osoby 30-50 let: {len(prescriptions_for_30_50)}")


analysis_included_30_50_vax_persons = [
    p
    for p in persons
    if p.vaccines and p.age_cohort == AgeCohort.BETWEEN_30_AND_50 and p.died_at is None
]
print(
    f"Počet očkovaných osob pro analýzu 30-50 let: {len(analysis_included_30_50_vax_persons)}"
)

prescriptions_for_30_50_vax_persons_2020_2023 = [
    1
    for p in persons
    if p.vaccines
    for prescription in p.prescriptions
    if prescription.date >= datetime(2020, 1, 1).date()
    and prescription.date <= datetime(2023, 1, 1).date()
    and prescription.age_cohort_at_prescription == AgeCohort.BETWEEN_30_AND_50
]
print(
    f"Počet předpisů pro očkované osoby 30-50 let v letech 2020-2023: {len(prescriptions_for_30_50_vax_persons_2020_2023)}"
)


number_of_first_prescriptions_in_2020_2023 = [
    1
    for p in persons
    if p.prescriptions
    and min(prescription.date for prescription in p.prescriptions)
    >= datetime(2020, 1, 1).date()
    and min(prescription.date for prescription in p.prescriptions)
    <= datetime(2023, 1, 1).date()
]
print(
    f"Počet prvopředpisů v letech 2020-2023: {len(number_of_first_prescriptions_in_2020_2023)}"
)

Počet osob s očkováním: 765972
Počet osob bez očkování a bez předpisů: 630500
Počet předpisů pro osoby 30-50 let: 735528
Počet očkovaných osob pro analýzu 30-50 let: 257909
Počet předpisů pro očkované osoby 30-50 let v letech 2020-2023: 180884
Počet prvopředpisů v letech 2020-2023: 124820


In [ ]:
no_of_corts = 0
no_of_corts_in_2020_2023 = 0
for person in cpzp_persons:
    if not person.vaccines:
        continue
    for prescription in person.prescriptions:
        if not prescription.age_cohort_at_prescription == AgeCohort.BETWEEN_30_AND_50:
            continue
        no_of_corts += 1
        if (
            prescription.date >= datetime(2020, 1, 1).date()
            and prescription.date <= datetime(2023, 1, 1).date()
        ):
            no_of_corts_in_2020_2023 += 1

print(no_of_corts)
print(no_of_corts_in_2020_2023)

517853
180884
